In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [123]:
import os
import pandas as pd
import numpy as np
import regex as re
from copy import deepcopy
import math
import pickle
import random

random.seed()

root = "/content/drive/MyDrive/Datasets"
os.chdir(root)

df = pd.read_csv('A1_dataset.csv', sep=',', header=None)
master_dataset_a = df.values[1:]
dataset_a = deepcopy(master_dataset_a)
print(dataset_a)

[['0' 'Fri Jun 05 14:26:50 2009' 'About to get threaded and scared ']
 ['1' 'Thu May 14 10:13:55 2009'
  '@awaisnaseer I like Shezan Mangooo too!!! I had one yesterday ']
 ['1' 'Fri Jun 05 21:02:20 2009'
  'worked on my car after work. showering then going to bed. sooooooooooo tired. sparrow signing out  &lt;Cowboy Up&gt;']
 ...
 ['1' 'Fri May 01 22:00:42 2009'
  'Just returned from the forest! Sarah (my merchy) lost her keys in the woods! what a day! ']
 ['1' 'Sun Jun 07 02:09:46 2009'
  'is proud of her dad and his piece of work. ( http://www.hakkastudy.in.th/) keep it up papa ']
 ['0' 'Fri May 22 04:49:37 2009'
  'Just woke up, gonna eat pizza for breakfast. Also, i have a dentist appointment so im gonna get checked out early. illl miss band though. ']]


In [3]:
# Remove CMU ARK
!rm -rf /content/drive/MyDrive/Datasets/lib/*

In [4]:
os.chdir(root)
# !wget https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/ark-tweet-nlp/ark-tweet-nlp-0.3.2.tgz

In [5]:
os.chdir(root)
# !ls ./lib/ark-tweet-nlp-0.3.2
!pwd
# !mkdir ./lib
!tar -xvzf /content/drive/MyDrive/Datasets/ark-tweet-nlp-0.3.2.tgz --directory=./lib

/content/drive/MyDrive/Datasets
ark-tweet-nlp-0.3.2/
ark-tweet-nlp-0.3.2/ark-tweet-nlp-0.3.2.jar
ark-tweet-nlp-0.3.2/data/
ark-tweet-nlp-0.3.2/docs/
ark-tweet-nlp-0.3.2/examples/
ark-tweet-nlp-0.3.2/LICENSE.txt
ark-tweet-nlp-0.3.2/README.txt
ark-tweet-nlp-0.3.2/runTagger.sh
ark-tweet-nlp-0.3.2/scripts/
ark-tweet-nlp-0.3.2/twokenize.sh
ark-tweet-nlp-0.3.2/scripts/paired_stats.r
ark-tweet-nlp-0.3.2/scripts/phraseomatic.py
ark-tweet-nlp-0.3.2/scripts/show.py
ark-tweet-nlp-0.3.2/scripts/simple_tests.sh
ark-tweet-nlp-0.3.2/scripts/toconll.py
ark-tweet-nlp-0.3.2/examples/bill_nye_tho.txt
ark-tweet-nlp-0.3.2/examples/casual.txt
ark-tweet-nlp-0.3.2/examples/example_tweets.txt
ark-tweet-nlp-0.3.2/examples/README.txt
ark-tweet-nlp-0.3.2/examples/tagged_tweets_expected.txt
ark-tweet-nlp-0.3.2/examples/tweets.jsonline
ark-tweet-nlp-0.3.2/docs/annot_guidelines.md
ark-tweet-nlp-0.3.2/docs/changelog.txt
ark-tweet-nlp-0.3.2/docs/hacking.txt
ark-tweet-nlp-0.3.2/docs/nathan_notes.md
ark-tweet-nlp-0.3.2/

In [6]:
!sudo apt-get install swig
!pip install jamspell
os.chdir(root)
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz 
!tar -xvf  'en.tar.gz' -C './'
!ls

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 12 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 0s (2,632 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fr

In [86]:
import collections
import jamspell

os.chdir(root)

corrector = jamspell.TSpellCorrector()
assert corrector.LoadLangModel('en.bin')  #load model

def jamSpellCorrect(text): #this is the one being used
    return corrector.FixFragment(text)

def createTextFile(dataset, text_path):
    f = open(text_path, "w")
    # f_pos = open(text_path_pos, "w")
    # f_neg = open(text_path_neg, "w")
    for label, _, sentence in dataset:
        f.write(sentence+'\n')
        # if label == "0":
        #     f_neg.write(sentence+"'\n")
        # elif label == "1":
        #     f_pos.write(sentence+"'\n")
    f.close()
    # f_pos.close()
    # f_neg.close()


# Takes in dataset and returns preProcessed dataset
def preProcessing(dataset, text_path):
    # text_path = dataset_path+".txt"2
    # text_path_pos = dataset_path+"_pos.txt"
    # text_path_neg = dataset_path+"_neg.txt"
    createTextFile(dataset, text_path)


    os.chdir(root+'/lib/ark-tweet-nlp-0.3.2')
    os.system(f"./twokenize.sh {text_path} > {root}/tokenized.txt")

    text_path = root+"/tokenized.txt"

    f = open(text_path, "r")
    fw = open(root + "/tokenizedad.txt", "w")
    dataset_proc = []
    preproc_sentences = []

    for label, _, sentence in dataset:
        temp = f.readline()
        # print(temp, end="")
        temp = temp[:-1 - len(sentence)-1].lower()
        # print(temp)
        # Spell Correction
        temp = jamSpellCorrect(temp)
        # preproc_sentences.append(temp)
        fw.write(temp + '\n')
        dataset_proc.append((temp, label))

    bigram_count = collections.defaultdict(int)
    unigram_count = collections.defaultdict(int)
    start_words = []

    total_sentences = 0
    total_words = 0

    for sentence, label in dataset_proc:
        proc_sentence = ""
        sentence = sentence.split()
        if len(sentence) == 0:
            continue

        total_sentences += 1
        # total

        for i in range(len(sentence)-1):
            if(sentence[i][0] == "#" or sentence[i][0] == "@" or sentence[i][0] == "-" or sentence[i][0][:4] == "http"):
                continue
            bigram_count[(sentence[i], sentence[i+1], label)] += 1
            # unigram_count[(sentence[i], label)] += 1
            # if(sentence[i] )
            unigram_count[(sentence[i], label)] = unigram_count[(sentence[i], label)] + 1
            proc_sentence += " " + sentence[i]
            total_words += 1
        
        if(not (sentence[-1][0] == "#" or sentence[-1][0] == "@" or sentence[-1][0] == "-" or sentence[-1][0][:4] == "http")):
            unigram_count[(sentence[-1], label)] += 1 
            proc_sentence += " " + sentence[-1]
            total_words += 1
        
        if(not (sentence[0][0] == "#" or sentence[0][0] == "@" or sentence[0][0] == "-" or sentence[0][0][:4] == "http")):
            start_words.append(sentence[0])
        
        preproc_sentences.append(proc_sentence)
    
    word_tokens = [i[0] for i in unigram_count.keys()]
    word_tokens = list(set(word_tokens))
    start_words = list(set(start_words))

    print("Average sentence word length: ", total_words/total_sentences)

    f.close()
    fw.close()

    # print("start words: ", len(start_words))
    
    return (word_tokens, unigram_count, bigram_count, start_words, np.array(preproc_sentences))
    


# createTextFile(dataset_a[1:], root + "/dataset_pos_a.txt", root + "/dataset_neg_a.txt")
# _, unig, _, _ = preProcessing(dataset_a, root+"/dataset_a.txt")
# os.chdir(root)

# print(unig[('and', '0')])
# print(unig[('and', '1')])

# for key in unig.keys():
#     print(key)
#     print(unig[key])





In [8]:
os.chdir(root+'/lib/ark-tweet-nlp-0.3.2')
!pwd
!ls
# !chmod u+x twokenize.sh
# os.system("./twokenize.sh examples/example_tweets.txt")
# !./twokenize.sh --help
# !rm -rf ./*

/content/drive/MyDrive/Datasets/lib/ark-tweet-nlp-0.3.2
ark-tweet-nlp-0.3.2.jar  docs	   LICENSE.txt	runTagger.sh  twokenize.sh
data			 examples  README.txt	scripts


In [9]:
total_pos_sentiment = 0
total_neg_sentiment = 0
for i,_,_ in dataset_a:
    if i == "1":
        total_pos_sentiment += 1
    else:
        total_neg_sentiment += 1
print(total_pos_sentiment, total_neg_sentiment)

2287 2000


In [10]:
def probBetaAndSmoothingOld(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k):
    """
    Returns a list of top_k choices.
    Beta is a probability function based on sentiment.
    Smooting is Laplace.
    """

    # print(dict_unigram_count[('and','1')])
    
    pos_sentiment = "1"
    neg_sentiment = "0"
    total_occ_word_i = dict_unigram_count[(word_i, neg_sentiment)] + dict_unigram_count[(word_i, pos_sentiment)]
    # print(dict_unigram_count)

    list_top_k = list()
    for word_i_post in unique_tokens:
        # print(word_i_post)
        total_occ_word_i_post_after_word_i = dict_bigram_count[(word_i, word_i_post, neg_sentiment)]+dict_bigram_count[(word_i, word_i_post, pos_sentiment)]

        if sentiment == "1":
            Beta = (dict_unigram_count[(word_i_post, sentiment)]+1)/(2*total_pos_sentiment)
        else:
            Beta = (dict_unigram_count[(word_i_post, sentiment)]+1)/(2*total_neg_sentiment)
        # Beta = 1
        list_top_k.append([word_i_post, ((total_occ_word_i_post_after_word_i+1)/(total_occ_word_i+len(unique_tokens)))*Beta])
    
    list_top_k.sort(key = lambda row: row[1], reverse = True)
    list_top_k = list_top_k[:top_k]
    return list_top_k

In [11]:
def probBetaAndSmoothing(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k):
    """
    Returns a list of top_k choices.
    Beta is a probability function based on sentiment.
    Smooting is Laplace.
    """

    # print(dict_unigram_count[('and','1')])
    
    pos_sentiment = "1"
    neg_sentiment = "0"
    total_occ_word_i = dict_unigram_count[(word_i, neg_sentiment)] + dict_unigram_count[(word_i, pos_sentiment)]
    # print(dict_unigram_count)

    list_top_k = list()
    for word_i_post in unique_tokens:
        # print(word_i_post)
        total_occ_word_i_post_after_word_i = dict_bigram_count[(word_i, word_i_post, neg_sentiment)]+dict_bigram_count[(word_i, word_i_post, pos_sentiment)]

        # if sentiment == "1":
        Beta = (dict_unigram_count[(word_i_post, sentiment)]+1)/(dict_unigram_count[(word_i_post, pos_sentiment)]+dict_unigram_count[(word_i_post, neg_sentiment)]+len(unique_tokens))
        # else:
        #     Beta = (dict_unigram_count[(word_i_post, sentiment)]+1)/(dict_unigram_count[(word_i_post, pos_sentiment)]+dict_unigram_count[(word_i_post, neg_sentiment)])
        # Beta = 1
        list_top_k.append([word_i_post, ((total_occ_word_i_post_after_word_i+1)/(total_occ_word_i+len(unique_tokens)))*Beta])
    
    list_top_k.sort(key = lambda row: row[1], reverse = True)
    list_top_k = list_top_k[:top_k]
    return list_top_k

In [12]:
def probBetaAndSmoothingPrevSentiment(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k):
    """
    Returns a list of top_k choices.
    Beta is a probability function based on sentiment.
    Smooting is Laplace.
    """

    # print(dict_unigram_count[('and','1')])
    
    pos_sentiment = "1"
    neg_sentiment = "0"
    total_occ_word_i = dict_unigram_count[(word_i, neg_sentiment)] + dict_unigram_count[(word_i, pos_sentiment)]
    word_i_sentiment = None
    total_occ_word_i_pos = dict_unigram_count[(word_i, pos_sentiment)]
    total_occ_word_i_neg = dict_unigram_count[(word_i, neg_sentiment)]
    prob_word_i_pos = (1 + total_occ_word_i_pos)/(2 + total_occ_word_i)
    prob_word_i_neg = (1 + total_occ_word_i_neg)/(2 + total_occ_word_i)
    
    if (prob_word_i_pos > prob_word_i_neg):
        word_i_sentiment = pos_sentiment
    else:
        word_i_sentiment = neg_sentiment


    list_top_k = list()
    for word_i_post in unique_tokens:
        # print(word_i_post)
        total_occ_word_i_post_after_word_i = dict_bigram_count[(word_i, word_i_post, neg_sentiment)]+dict_bigram_count[(word_i, word_i_post, pos_sentiment)]

        prob_word_i_word_i_post = (1 + total_occ_word_i_post_after_word_i)/((2*len(unique_tokens)) + total_occ_word_i)

        total_occ_word_i_post_pos = dict_unigram_count[(word_i_post, pos_sentiment)]
        total_occ_word_i_post_neg = dict_unigram_count[(word_i_post, neg_sentiment)]
        total_occ_word_i_post = total_occ_word_i_post_pos + total_occ_word_i_post_neg


        # prob_word_i_post = 0
        if word_i_sentiment == pos_sentiment:
            prob_word_i_post = (1 + total_occ_word_i_post_pos)/(2 + total_occ_word_i_post)
        else:
            prob_word_i_post = (1 + total_occ_word_i_post_neg)/(2 + total_occ_word_i_post)

        prob_word_i_sentiment_word_i_post  = (1 + prob_word_i_post)/(2 + total_occ_word_i_post)

        final_prob = prob_word_i_word_i_post*prob_word_i_sentiment_word_i_post

        list_top_k.append([word_i_post, final_prob])
    
    list_top_k.sort(key = lambda row: row[1], reverse = True)
    list_top_k = list_top_k[:top_k]
    return list_top_k

In [54]:
def probBetaAndSmoothing3(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k):
    """
    Returns a list of top_k choices.
    Beta is a probability function based on sentiment.
    Smooting is Laplace.
    """

    # print(dict_unigram_count[('and','1')])
    
    pos_sentiment = "1"
    neg_sentiment = "0"
    total_occ_word_i = dict_unigram_count[(word_i, neg_sentiment)] + dict_unigram_count[(word_i, pos_sentiment)]
    # print(dict_unigram_count)

    list_top_k = list()
    for word_i_post in unique_tokens:
        # print(word_i_post)
        total_occ_word_i_post_after_word_i = dict_bigram_count[(word_i, word_i_post, neg_sentiment)]+dict_bigram_count[(word_i, word_i_post, pos_sentiment)]

        if sentiment == "1":
            Beta = (dict_unigram_count[(word_i_post, pos_sentiment)] - dict_unigram_count[(word_i_post, neg_sentiment)])/(dict_unigram_count[(word_i_post, pos_sentiment)] + dict_unigram_count[(word_i_post, neg_sentiment)])
            Beta = Beta/2 + 1/2 #Bringing to range [0,1]
        else:
            Beta = (dict_unigram_count[(word_i_post, neg_sentiment)] - dict_unigram_count[(word_i_post, pos_sentiment)])/(dict_unigram_count[(word_i_post, pos_sentiment)] + dict_unigram_count[(word_i_post, neg_sentiment)])
            Beta = Beta/2 + 1/2 #Bringing to range [0,1]
        # Beta = 1
        list_top_k.append([word_i_post, ((total_occ_word_i_post_after_word_i+1)/(total_occ_word_i+len(unique_tokens))) + 2e-10*Beta])
    
    list_top_k.sort(key = lambda row: row[1], reverse = True)
    list_top_k = list_top_k[:top_k]
    return list_top_k

In [14]:
def masterProbBetaAndSmoothing(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k, smoothing_choice):
    if smoothing_choice == 0:
        return probBetaAndSmoothing(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k)
    elif smoothing_choice == 1:
        return probBetaAndSmoothingOld(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k)
    elif smoothing_choice == 2:
        return probBetaAndSmoothingPrevSentiment(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k)
    elif smoothing_choice == 3:
        return probBetaAndSmoothing2(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k)
    elif smoothing_choice == 4:
        return probBetaAndSmoothing3(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k)



In [125]:
import random

os.chdir(root)

# generate sentences
def genSentencesAndPerplexityCalculator(no_of_sentences, sentiment, dataset, min_word_limit, max_word_limit):
    word_tokens, unigram_count, bigram_count, start_words,_ = preProcessing(dataset, root+"/dataset_a.txt")
    random.seed()
    end_tokens = ['.', '!', '?']
    end_tokens = []
    gen_sentences = []
    perplexities = []

    for i in range(no_of_sentences):
        conditioning = start_words[random.randint(0, len(start_words) - 1)]
        sentence = "" + conditioning
        current_word = conditioning
        no_of_words = 1

        perplexity = 1/len(start_words)

        while no_of_words < max_word_limit and (current_word not in end_tokens or no_of_words < 15): 
            # found_next_word = False
            next_word = None
            prob_next_word = 0
            possible_words = masterProbBetaAndSmoothing(word_tokens, unigram_count, bigram_count, current_word, sentiment, 20, 3)
            # print(possible_words)

            if no_of_words > min_word_limit:
                next_word, prob_next_word  = possible_words[random.randint(0,3)]
            else:
                for i in range(random.randint(0,5), len(possible_words)):
                    if possible_words[i][0] not in end_tokens and possible_words[i] != current_word:
                        next_word, prob_next_word = possible_words[i]
                        break
            
            if next_word == None:
                break

            perplexity *= prob_next_word

            no_of_words += 1
            sentence += " " + next_word
            current_word = next_word
        
        perplexity = math.pow(1/perplexity, 1/no_of_words)
        
        # print(sentence)
        # print(f"perplexity: {perplexity}")

        gen_sentences.append(sentence)
        perplexities.append(perplexity)
    
    return gen_sentences, perplexities





In [145]:
def probBetaAndSmoothing2(unique_tokens, dict_unigram_count, dict_bigram_count, word_i, sentiment, top_k):
    """
    Returns a list of top_k choices.
    Beta is a probability function based on sentiment.
    Smooting is Laplace.
    """

    # print(dict_unigram_count[('and','1')])
    
    pos_sentiment = "1"
    neg_sentiment = "0"
    total_occ_word_i = dict_unigram_count[(word_i, neg_sentiment)] + dict_unigram_count[(word_i, pos_sentiment)]
    # print(dict_unigram_count)

    list_top_k = list()
    sum_prob = 0

    for word_i_post in unique_tokens:
        # print(word_i_post)
        total_occ_word_i_post_after_word_i = dict_bigram_count[(word_i, word_i_post, neg_sentiment)]+dict_bigram_count[(word_i, word_i_post, pos_sentiment)]

        if sentiment == "1":
            Beta = (dict_unigram_count[(word_i_post, pos_sentiment)]+1)/(dict_unigram_count[(word_i_post, neg_sentiment)]+1)
        else:
            Beta = (dict_unigram_count[(word_i_post, neg_sentiment)]+1)/(dict_unigram_count[(word_i_post, pos_sentiment)]+1)
        # Beta = 0
        Alpha = 1.3e-5
        list_top_k.append([word_i_post, ((total_occ_word_i_post_after_word_i+1)/(total_occ_word_i+len(unique_tokens)))+Alpha*Beta])
        sum_prob += list_top_k[-1][1]
    
    list_top_k.sort(key = lambda row: row[1], reverse = True)

    # sum = 0
    # for i in list_top_k:
    #     sum += i[1]
    # print(sum)

    for i in list_top_k:
        i[1] = i[1]/sum_prob
    
    # sum = 0    
    # for i in list_top_k:
    #     sum += i[1]
    
    # print(sum)
    
    list_top_k = list_top_k[:top_k]
    return list_top_k

In [146]:
gen_sent_pos, perxs_pos = genSentencesAndPerplexityCalculator(350, "1", dataset_a, 7, 20)
print(gen_sent_pos)

Average sentence word length:  14.29344529974341
["bout you ! but it's not sure , so sad . i was so i love it . but i'm", "we've congrats congrats ! it , so i love the last time to the day of it ! you !", "alkaline trio congrats !! lol . it's been there , so sad day of my mom is not sure you", 'correction congrats !! and i have to see the day ! i love the last one of my friends ,', 'alaaaa ...... i love you ! it ! and now i love the day of my friends , and a', 'lots asked him . but i was the last one day to the last one . i love it !', 'ladies and it ! i am in my friends are so sad i am so much ! i am i', "should i love the way . i'm not going out with a new glasses ! i have the same time", 'laying out with a few days ! you can get a great time to be in my phone calls !', 'fantastic movie for you are the same thing is not a great . but it is the best of the', "ngl congrats , i'm so bad for a new moon congrats enjoy your time for the day to see you", "supernatural asked congrats enjoy y

In [147]:
gen_sent_neg, perxs_neg = genSentencesAndPerplexityCalculator(150, "0", dataset_a, 7, 20)
print(gen_sent_neg)
print("Avergage perplexity is:",(sum(perxs_pos)+sum(perxs_neg))/(len(perxs_pos)+len(perxs_neg)))

Average sentence word length:  14.29344529974341
['class sick for some shit died ! but it . but i love you ! and a little sad i', 'happy mothers sick of a little sad about to the best friend . but i love it was just finished', "untitled sucks sad day of the same !! my head sick of it . it's not a few sick of", "plus poor sucks , i'm so , the best friend is a great . but its so i am not", 'sorry i can do you ! i am in a good night ! i am not going out the best', 'back from the best friend is not going out in bed , so bad sick for you have been trying', "your time i love it will make the best of a little bit sad . i'm not a few days", 'humidity anymore sucks broke broke broke sad that is going to see my phone calls sucks sad . but it', "june sick sad about it . it's a great . it's a few days ! i love that i was", 'my friend is it will not sure you are not going out of a few sick for me too .', "idk what to the day ? i'm going out with me too ! you . i'm going back to the", "fantastic bro

In [19]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [74]:
def checkSentimentAcc(gen_sentences, sentiment):
    sentimentAnalyzer = SentimentIntensityAnalyzer()
    total_count = len(gen_sentences)
    correct_count = 0
    for text in gen_sentences:
        score = sentimentAnalyzer.polarity_scores(text)
        if(sentiment == "0"):
            score['compound'] = score['compound']*(-1)
        if score['compound'] >= 0.05:
            correct_count += 1
    
    print(f"Accuracy {sentiment}: {correct_count}/{total_count}: {correct_count*100/total_count}%")

In [148]:
checkSentimentAcc(gen_sent_neg, "0")
checkSentimentAcc(gen_sent_pos, "1")

Accuracy 0: 96/150: 64.0%
Accuracy 1: 277/350: 79.14285714285714%


In [135]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
def train_and_evaluate(train_sentences, train_labels, test_sentences, test_labels):
    '''
    parameters:
    train_sentences : list of training sentences
    train_labels : list of training labels
    test_sentences : list of test sentences
    test_labels : list of test labels
    output:
    accuracy : accuracy of the test set
    '''
    # Model building
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    # Training the model with the training data
    model.fit(train_sentences, train_labels)
    # Predicting the test data categories
    predicted_test_labels = model.predict(test_sentences)
    return accuracy_score(test_labels, predicted_test_labels)

In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

def train_and_evaluate(train_sentences, train_labels, test_sentences, test_labels):
    '''
    parameters:
    train_sentences : list of training sentences
    train_labels : list of training labels
    test_sentences : list of test sentences
    test_labels : list of test labels
    output:
    accuracy : accuracy of the test set
    '''
    # Model building
    # model = make_pipeline(TfidfVectorizer(), StandardScaler(), SVC(gamma='auto'))
    model = make_pipeline(TfidfVectorizer(), DecisionTreeClassifier(max_depth=50))
    # Training the model with the training data
    model.fit(train_sentences, train_labels)
    # Predicting the test data categories
    predicted_test_labels = model.predict(test_sentences)
    return accuracy_score(test_labels, predicted_test_labels)

In [149]:
os.chdir(root)

df2 = pd.read_csv('A2_test_dataset.csv', sep=',', header=None)
master_dataset_test = df2.values[1:]
dataset_test = deepcopy(master_dataset_test)
print(dataset_test)

[['1' 'Fri May 29 22:24:26 2009'
  '@mileycyrus cheer up miley whats wrong? ']
 ['1' 'Sun Jun 07 01:37:36 2009'
  'Just got back in from The Belcourt. Saw &quot;The Fifth Element.&quot; It was awesome. Now uploading photos from Friday and tonight! ']
 ['1' 'Wed May 13 23:41:18 2009' 'http://bit.ly/IQPPD  with video ']
 ...
 ['0' 'Mon Jun 01 05:14:30 2009'
  'Sick--or allergytastic (not sure which!)--just in time for Miss Saigon auditions. Splendid. ']
 ['0' 'Sun Jun 07 08:16:06 2009' 'headache  ... but sooooo worth it haha']
 ['1' 'Tue Jun 02 05:33:31 2009'
  "Revising Byron in me garden, it's well hot!! Canto 1 &amp; 2 done, just 3 &amp; 4 to go woo "]]


In [158]:
print(len(dataset_a))
print(train_and_evaluate(dataset_a[:,2], dataset_a[:,0], dataset_test[:,2], dataset_test[:,0]))
print("Accuracy A and test:",train_and_evaluate(preProcessing(dataset_a, root+"/dataset_a_preproc.txt")[4], dataset_a[:,0], preProcessing(dataset_test, root+"/dataset_test_preproc.txt")[4], dataset_test[:,0]))

4287
0.8555900621118012
Average sentence word length:  14.29344529974341
Average sentence word length:  14.51086956521739
Accuracy A and test: 0.8649068322981367


In [159]:
def generateDatasetB(gen_sentences):
    dataset_b = []
    sentimentAnalyzer = SentimentIntensityAnalyzer()
    for text in gen_sentences:
        score = sentimentAnalyzer.polarity_scores(text)
        if score['compound'] >= 0: #since we only have positive and negative labels
            label = "1"
        else:
            label = "0"
        dataset_b.append(np.array([label, "None", text]))
    return np.array(dataset_b)

dataset_b = np.concatenate((dataset_a,generateDatasetB(gen_sent_pos+gen_sent_neg)), axis = 0)
print("dataset b shape: ", dataset_b.shape)
# print(len(preProcessing(dataset_b, root+"/dataset_b_preproc.txt")[4][:]))
# print(preProcessing(dataset_b, root+"/dataset_b_preproc.txt")[4].shape)

print(train_and_evaluate(dataset_b[:,2], dataset_b[:,0], dataset_test[:,2], dataset_test[:,0]))
print("Accuracy B and test:", train_and_evaluate(preProcessing(dataset_b, root+"/dataset_b_preproc.txt")[4], dataset_b[:,0], preProcessing(dataset_test, root+"/dataset_test_preproc.txt")[4], dataset_test[:,0]))

dataset b shape:  (4787, 3)
0.8633540372670807
Average sentence word length:  14.889492375182787
Average sentence word length:  14.51086956521739
Accuracy B and test: 0.8664596273291926


In [162]:
os.mkdir(root+"/NLP_Assignment_2")
os.chdir(root+"/NLP_Assignment_2")
_,_,dict_bigram_count,_,_ = preProcessing(dataset_a, root+"/dataset_a.txt")

Average sentence word length:  14.29344529974341


In [163]:
with open(root+"/NLP_Assignment_2/smoothed_bigram_LM.pickle", "wb") as file:
    pickle.dump(dict_bigram_count, file, pickle.HIGHEST_PROTOCOL)
with open(root+"/NLP_Assignment_2/smoothed_bigram_LM.pickle", "rb") as file:
    loaded_dict = pickle.load(file)
print(loaded_dict)

defaultdict(<class 'int'>, {('about', 'to', '0'): 7, ('to', 'get', '0'): 43, ('get', 'threaded', '0'): 1, ('threaded', 'and', '0'): 1, ('and', 'scared', '0'): 1, ('i', 'like', '1'): 13, ('like', 'shezan', '1'): 1, ('shezan', 'mango', '1'): 1, ('mango', 'too', '1'): 1, ('too', '!!!', '1'): 1, ('!!!', 'i', '1'): 5, ('i', 'had', '1'): 15, ('had', 'one', '1'): 1, ('one', 'yesterday', '1'): 1, ('worked', 'on', '1'): 1, ('on', 'my', '1'): 24, ('my', 'car', '1'): 3, ('car', 'after', '1'): 1, ('after', 'work', '1'): 2, ('work', '.', '1'): 9, ('.', 'showering', '1'): 1, ('showering', 'then', '1'): 1, ('then', 'going', '1'): 1, ('going', 'to', '1'): 67, ('to', 'bed', '1'): 17, ('bed', '.', '1'): 8, ('.', 'sooooooooooo', '1'): 1, ('sooooooooooo', 'tired', '1'): 1, ('tired', '.', '1'): 5, ('.', 'sparrow', '1'): 1, ('sparrow', 'signing', '1'): 1, ('signing', 'out', '1'): 1, ('out', '<', '1'): 1, ('<', 'cowboy', '1'): 1, ('cowboy', 'up', '1'): 1, ('up', '>', '1'): 1, ('actually', 'we', '1'): 1, ('we

In [164]:
def top4Bigrams(dict_bigram):
    pos_sentiment = "1"
    neg_sentiment = "0"
    top_4_bigrams_count = [0,0,0,0]
    top_4_bigrams = [[None, None], [None, None], [None, None], [None, None]]
    sum = 0
    for j in range(len(top_4_bigrams)):
        for word, word_post, s in list(dict_bigram):
            sum = dict_bigram[(word, word_post, pos_sentiment)]+dict_bigram[(word, word_post, neg_sentiment)]
            flag = False
            for i in range(len(top_4_bigrams)):
                if word == top_4_bigrams[i][0] and word_post == top_4_bigrams[i][1]:
                    flag = True
                    break
            if flag:
                continue
            if sum > top_4_bigrams_count[j]:
                top_4_bigrams_count[j] = sum
                top_4_bigrams[j] = [word, word_post]
    return top_4_bigrams, top_4_bigrams_count

print(top4Bigrams(dict_bigram_count))

([['.', 'i'], [',', 'i'], ['i', 'have'], ['in', 'the']], [176, 127, 120, 119])


In [ ]:
os.chdir(root+"/NLP_Assignment_2")
# a = generateDatasetB(gen_sent_pos+gen_sent_neg)
# print(a)
pd.DataFrame(generateDatasetB(gen_sent_pos+gen_sent_neg)).to_csv("sentences.csv")
df3 = pd.read_csv('sentences.csv', sep=',', header=None)
print(df3.values[1:,1:])

In [165]:
print("Negative sentences:")
for i in range(5):
    print(gen_sent_neg[i])
print("\nPostive sentences:")
for i in range(5):
    print(gen_sent_pos[i])

Negative sentences:
class sick for some shit died ! but it . but i love you ! and a little sad i
happy mothers sick of a little sad about to the best friend . but i love it was just finished
untitled sucks sad day of the same !! my head sick of it . it's not a few sick of
plus poor sucks , i'm so , the best friend is a great . but its so i am not
sorry i can do you ! i am in a good night ! i am not going out the best

Postive sentences:
bout you ! but it's not sure , so sad . i was so i love it . but i'm
we've congrats congrats ! it , so i love the last time to the day of it ! you !
alkaline trio congrats !! lol . it's been there , so sad day of my mom is not sure you
correction congrats !! and i have to see the day ! i love the last one of my friends ,
alaaaa ...... i love you ! it ! and now i love the day of my friends , and a
